In [1]:
%load_ext autoreload
%autoreload 2 

import os; os.chdir(f'{os.getenv("MT_SOURCE")}'); os.getcwd()

'/Users/Play/Code/AI/master-thesis/src'

In [2]:
import pathlib as pl
import json

import pandas as pd

import constants as ct
import helpers as hp
from postpro.visualisers import visualiser_class as vc, visualiser_ae as va, visualiser_vae as vv 

In [13]:
SIZE = ['size']
ACC1 = ['train_acc_1', 'dev_acc_1', 'test_acc_1']
ACC5 = ['train_acc_5', 'dev_acc_5', 'test_acc_5']
CE_LOSS = ['train_ce_loss', 'dev_ce_loss', 'test_ce_loss']
MSE_LOSS = ['train_mse_loss', 'dev_mse_loss', 'test_mse_loss']
KLD_LOSS = ['train_kld_loss', 'dev_kld_loss', 'test_kld_loss']

In [9]:
def merge_evaluations(runs_dir: pl.Path) -> pd.DataFrame:
    run_paths = sorted(runs_dir.glob("**/run.json"))
    metrics_paths = sorted(runs_dir.glob("**/results.json"))
    dfs = []
    for run_path, results_path in zip(run_paths, metrics_paths):
        with open(run_path.as_posix()) as run_file, open(results_path.as_posix())  as results_file:
            run = json.load(run_file)
            results = {k: [v] for k,v in json.load(results_file).items()}
            results.update({
                'model': '_'.join(run['name'].split('/')[-2:]),
                'size': run['model']['size']
            })
            dfs.append(pd.DataFrame.from_dict(results).set_index('model'))
    _df = pd.concat(dfs, axis=0, sort=False).sort_index()
    for col in _df:
        if 'acc' in col:
            _df[col] = round(_df[col] * 100, 2)
    
    return _df

In [10]:
df = merge_evaluations(pl.Path(ct.HMDB_RUN_DIR_1))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, tarn_aelarge to tarn_vaelarge
Data columns (total 19 columns):
train_acc_1         3 non-null float64
train_acc_5         3 non-null float64
train_ce_loss       3 non-null float64
train_mse_loss      2 non-null float64
train_total_loss    2 non-null float64
dev_acc_1           3 non-null float64
dev_acc_5           3 non-null float64
dev_ce_loss         3 non-null float64
dev_mse_loss        2 non-null float64
dev_total_loss      2 non-null float64
test_acc_1          3 non-null float64
test_acc_5          3 non-null float64
test_ce_loss        3 non-null float64
test_mse_loss       2 non-null float64
test_total_loss     2 non-null float64
size                3 non-null object
train_kld_loss      1 non-null float64
dev_kld_loss        1 non-null float64
test_kld_loss       1 non-null float64
dtypes: float64(18), object(1)
memory usage: 480.0+ bytes


In [11]:
df[SIZE + ACC1 + ACC5]

,size,train_acc_1,dev_acc_1,test_acc_1,train_acc_5,dev_acc_5,test_acc_5
model,,,,,,,
tarn_aelarge,"10,543,011",84.38,3.12,6.25,96.88,9.38,25.00
tarn_class,"6,492,227",84.38,7.81,6.25,100.00,25.00,23.44
tarn_vaelarge,"10,674,083",90.62,9.38,3.12,100.00,25.00,28.12


In [21]:
df.loc[df.index.str.contains('class'), SIZE + CE_LOSS]

,size,train_ce_loss,dev_ce_loss,test_ce_loss
model,,,,
tarn_class,"6,492,227",0.341815,6.020304,6.540232


In [22]:
df.loc[df.index.str.contains('ae'), SIZE + CE_LOSS + MSE_LOSS]

,size,train_ce_loss,dev_ce_loss,test_ce_loss,train_mse_loss,dev_mse_loss,test_mse_loss
model,,,,,,,
tarn_aelarge,"10,543,011",0.385919,7.101376,7.246385,0.026363,0.031979,0.035490
tarn_vaelarge,"10,674,083",0.310771,7.208393,7.187164,0.053296,0.056069,0.064162


In [23]:
df.loc[df.index.str.contains('vae'), SIZE + CE_LOSS + MSE_LOSS + KLD_LOSS]

,size,train_ce_loss,dev_ce_loss,test_ce_loss,train_mse_loss,dev_mse_loss,test_mse_loss,train_kld_loss,dev_kld_loss,test_kld_loss
model,,,,,,,,,,
tarn_vaelarge,"10,674,083",0.310771,7.208393,7.187164,0.053296,0.056069,0.064162,0.334478,0.353053,0.358575
